<a href="https://colab.research.google.com/github/FranckCode/nlp-chatbot/blob/main/NLP_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nltk.download('punkt')

In [ ]:
import tensorflow as tf
print(tf.__version__)
tf.config.list_physical_devices('GPU')

In [ ]:
import numpy as np
import string
import re
import json
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
# Tokenization
tokenizer = Tokenizer(filters='', lower=True)

# Chemin vers le fichier JSON
dataset_file = 'dataset.json'

In [ ]:
# Prétraitement des données
def preprocess_text(text):

    # Convertir en minuscules
    text = text.lower()

    # Supprimer les caractères de ponctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Suppression des chiffres
    text = re.sub(r"\d+", "", text)

    # Tokenization
    tokens = word_tokenize(text)

    # Rejoindre les tokens en une seule chaîne de texte
    preprocessed_text = " ".join(tokens)

    return preprocessed_text


In [ ]:
def load_data_and_fit_modele():

    data=[]

    # Ouvrir le fichier JSON
    with open(dataset_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print(" ---> Début du Chargment des données")

    # Chargement des questions-réponses depuis la base de données
    questions = []
    answers = []

    # Charger le champ "question_response"
    questions_reponses = data['questions_responses']
    for item in questions_reponses:
        questions.append(item['question'])
        answers.append(item['response'])

    print(" ---x Fin du Chargment des données ")

    print(" ---> Début du Pré-traitement des données")

    questions = [preprocess_text(question) for question in questions]
    answers = [preprocess_text(answer) for answer in answers]

    print(" ---x Fin du Pré-traitement des données")

    print(" ---> Début de l'entrainement des données")

    tokenizer.fit_on_texts(questions + answers + ['[START]', '[END]'])

    print(" ---x Fin de l'entrainement des données")

    print(" ---> Début de la création des séquences")

    # Création des séquences d'index

    # Initialise une liste vide pour stocker les séquences d'entrée
    questions_sequences = []
    answers_sequences = []

    # Parcourt les réponses
    for question in questions:
        question_token_list = tokenizer.texts_to_sequences(question)[0]
        for answer in answers:
            # Convertit la réponse en une séquence d'index en utilisant le tokenizer
            answer_token_list = tokenizer.texts_to_sequences(['[START] ' + answer + ' [END]'])[0]
            # Parcourt la séquence d'index
            for i in range(1, len(answer_token_list)):
                # Crée une sous-séquence d'index à partir de la séquence d'index actuelle
                n_gram_sequence = answer_token_list[:i+1]
                answers_sequences.append(n_gram_sequence)
                questions_sequences.append(question_token_list)

    print(" ---x Fin de la création des séquences")

    print(" ---> Début du rembourrage des séquences")

    # Padding des séquences
    max_sequence_length = max(len(seq) for seq in questions_sequences + answers_sequences)
    questions_sequences = np.array(pad_sequences(questions_sequences, maxlen=max_sequence_length, padding='post'))
    answers_sequences = np.array(pad_sequences(answers_sequences, maxlen=max_sequence_length, padding='post'))

    print(" ---x Fin du rembourrage des séquences")

    print(" ---> Début de la création et l'entrainemment du modèle")

    # Modèle de chatbot
    vocab_size = len(tokenizer.word_index) + 1
    embedding_dim = 100
    hidden_units = 256

    # Encoder
    encoder_inputs = Input(shape=(max_sequence_length,))
    encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
    encoder_lstm = tf.keras.layers.Bidirectional(LSTM(hidden_units, return_sequences=True, return_state=True))
    _, forward_state_h, forward_state_c, backward_state_h, backward_state_c = encoder_lstm(encoder_embedding)
    state_h = tf.keras.layers.Concatenate()([forward_state_h, backward_state_h])
    state_c = tf.keras.layers.Concatenate()([forward_state_c, backward_state_c])
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = Input(shape=(max_sequence_length-1,))
    decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(hidden_units*2, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

    decoder_dense = Dense(vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Modèle complet
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # Compilation du modèle
    model.compile(optimizer=RMSprop(), loss='sparse_categorical_crossentropy')

    # Entraînement du modèle
    model.fit([questions_sequences, answers_sequences[:, :-1]], answers_sequences[:, 1:], epochs=1, batch_size=32)
    print(" ---x Fin de la création et l'entrainemment du modèle")


    print(" ---> Début de la sauvegarde du modèle")
    # Sauvegarde le max_sequence_length
    with open('max_sequence_length.json', 'w') as file:
        json.dump(max_sequence_length, file)

    # Sauvegarde du tokenizer
    with open('tokenizer.pickle', 'wb') as file:
        pickle.dump(tokenizer, file)

    # Sauvegarde le modèle entier et le max_sequence_length
    model.save('modele')
    print(" ---x Fin de la sauvegarde du modèle")


In [ ]:
load_data_and_fit_modele()

### **Test du modèle**

---



In [ ]:
model = ""
max_sequence_length = ""
tokenizer = ""

In [ ]:
def load_model():

    global model, max_sequence_length, tokenizer

    # Pour charger le modèle entier
    model = tf.keras.models.load_model('modele')

    # Chargement du max_sequence_length
    with open('max_sequence_length.json', 'r') as file:
        max_sequence_length = json.load(file)

    # Chargement du tokenizer
    with open('tokenizer.pickle', 'rb') as file:
        tokenizer = pickle.load(file)

In [ ]:
# Fonction pour générer une réponse à partir d'une question de l'utilisateur
def generate_response(question):
    # Prétraitement de la question de l'utilisateur
    preprocessed_question = preprocess_text(question)

    # Conversion en séquence d'index
    question_sequence = tokenizer.texts_to_sequences([preprocessed_question])
    question_sequence = pad_sequences(question_sequence, maxlen=max_sequence_length, padding='post')

    # Prédiction de la réponse
    answer_input = np.zeros((1, max_sequence_length-1))
    answer_input[0, 0] = tokenizer.word_index['[start]']

    for i in range(1, max_sequence_length-1):
        prediction = model.predict([question_sequence, answer_input])
        predicted_word_index = np.argmax(prediction[0, i-1, :])
        answer_input[0, i] = predicted_word_index
        # print(answer_input)

        if predicted_word_index == tokenizer.word_index['[end]']:
            break
    # Conversion de la séquence d'index en réponse textuelle
    answer = tokenizer.sequences_to_texts([answer_input[0, :i+1]])[0]
    answer = answer.replace('[start]', '').replace('[end]', '').strip()

    return answer


In [ ]:
load_model()
while True:
    user_question = input("\n Quel est votre question ? \t")
    response = generate_response(user_question)
    print("User:", user_question)
    print("Bot:", response)